<a href="https://colab.research.google.com/github/JoaoJuniorGrb/regressao_linear_tensorflow/blob/main/regress%C3%A3o_multipla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt

# Carregar base de dados
base = pd.read_csv('house_prices.csv')

#base.columns
#selecionar apenas as colunas que serão utilizadas
colunas_usadas = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']

base = base[colunas_usadas]

from sklearn.preprocessing import  MinMaxScaler


#normalização dos parametros de entrada
scaler_x = MinMaxScaler()
base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_x.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])



#normalização dos parametros de saida (por padrão sempre separar o scaler de x e de y, pesquisar depois o real motivo)
scaler_y = MinMaxScaler()
base[['price']] = scaler_y.fit_transform(base[['price']])


#separar o dataframe em dois sendo x as entradas e y a saida
x = base.drop('price', axis = 1)
y = base['price']

#dividir o treinamento 80% e teste 20%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Definir o modelo, foi selecionado a primeira camada de 64 neuronios, e segunda 32, e a saida de 1 que seria o preço
#relu é a função de ativação x= valor e 0 se negativo, o input shape nesse caso é esperado n colunas igual ao treinamento
modelo = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Uma saída para o preço
])

# Compilar o modelo, configurar os parametros de aprendizagem do modelo
modelo.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Treinar o modelo, executa o treinamento do modelo utilizando 50 treinamentos (vezes que percorre os dados),
# onde 20% será usado para validação e não treinamento, os dados são processados em lotes de 32 amostras para não utilizar tanta memoria
historico = modelo.fit(x_train, y_train, epochs=2000, validation_split=0.2, batch_size=1000)


resultado = modelo.evaluate(x_test, y_test)
print(f"Erro médio absoluto no teste: {resultado[1]}")


plt.plot(historico.history['loss'], label='Treinamento')
plt.plot(historico.history['val_loss'], label='Validação')
plt.xlabel('Épocas')
plt.ylabel('Erro Quadrático Médio (MSE)')
plt.legend()
plt.show()


# Previsões
y_pred = modelo.predict(x_test)

# Reverter a normalização
y_pred_original = scaler_y.inverse_transform(y_pred)
y_test_original = scaler_y.inverse_transform(y_test.values.reshape(-1, 1))


# Comparar resultados
for i in range(10):  # Mostrar as 5 primeiras previsões
    print(f"real: {y_test_original[i][0]:.2f}, Previsão: {y_pred_original[i][0]:.2f},real-previsão:{100*(float(y_test_original[i][0]) - float(y_pred_original[i][0]))/float(y_test_original[i][0]):.2f} %")


Epoch 1/2000


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0039 - mae: 0.0446 - val_loss: 0.0010 - val_mae: 0.0222
Epoch 2/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012 - mae: 0.0238 - val_loss: 7.4915e-04 - val_mae: 0.0193
Epoch 3/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 8.2780e-04 - mae: 0.0186 - val_loss: 6.2917e-04 - val_mae: 0.0168
Epoch 4/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7.7245e-04 - mae: 0.0172 - val_loss: 5.8043e-04 - val_mae: 0.0162
Epoch 5/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6.0085e-04 - mae: 0.0158 - val_loss: 5.4363e-04 - val_mae: 0.0154
Epoch 6/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6.0878e-04 - mae: 0.0155 - val_loss: 5.2435e-04 - val_mae: 0.0151
Epoch 7/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 5.3538e-04 - mae: 0.0147 - val_loss: 5.0090e-04 - val_mae: 0.0145
Epoch 8/2000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.9542e-04 - mae: 0.0142 - val_loss: 4.9356e-04 - val_mae: 0.0143
Epoch 9/2000
14/

KeyboardInterrupt: 

In [ ]:
# Comparar resultados
for i in range(10):  # Mostrar as 5 primeiras previsões
    print(f"real: {y_test_original[i][0]:.2f}, Previsão: {y_pred_original[i][0]:.2f}, real-previsão:{100*(float(y_test_original[i][0]) - float(y_pred_original[i][0]))/float(y_test_original[i][0]):.2f} %")


real: 365000.00, Previsão: 381939.25,real-previsão:-4.64 %
real: 865000.00, Previsão: 922891.38,real-previsão:-6.69 %
real: 1038000.00, Previsão: 1125253.75,real-previsão:-8.41 %
real: 1490000.00, Previsão: 2257185.50,real-previsão:-51.49 %
real: 711000.00, Previsão: 718689.75,real-previsão:-1.08 %
real: 211000.00, Previsão: 287563.47,real-previsão:-36.29 %
real: 790000.00, Previsão: 877787.00,real-previsão:-11.11 %
real: 680000.00, Previsão: 605290.44,real-previsão:10.99 %
real: 384500.00, Previsão: 522277.78,real-previsão:-35.83 %
real: 605000.00, Previsão: 617451.88,real-previsão:-2.06 %
